In [1]:
from hepdata_lib import RootFileReader
from hepdata_lib import Table
from hepdata_lib import Variable, Uncertainty
from hepdata_lib import Submission
def round_pair(array, n):
    newx = []
    for x in array:
        newx.append((round(x[0],n), round(x[1],n)))
    return newx

def round_list(array,n):
    newx = [ round(x,n) for x in array]
    return newx

Welcome to JupyROOT 6.22/02


In [2]:
reader = RootFileReader("Results/jet_shapes.root")
hjs_incl = reader.read_hist_1d('dr_data_incl_js_allpt_0_0')
hjs_b = reader.read_hist_1d('dr_data_bjtc_js_allpt_0_0')
hratio = reader.read_hist_1d('division_dr_data_bjtc_js_allpt_0_0')
hjs_incl_syst = reader.read_hist_1d('dr_data_incl_allpt_syst_err_0_0')
hjs_b_syst = reader.read_hist_1d('dr_data_bjtc_allpt_syt_err_0_0')
hratio_syst = reader.read_hist_1d('division_dr_data_bjtc_allpt_syt_err_0_0')

reader2 = RootFileReader("Results/particle_yield.root")
hpy_incl = reader2.read_hist_1d('js_incl_tot')
hpy_b = reader2.read_hist_1d('js_b_tot')
hdiff = reader2.read_hist_1d('diff_tot')
hpy_incl_syst = reader2.read_hist_1d('js_incl_err_tot')
hpy_b_syst = reader2.read_hist_1d('js_b_err_tot')
hdiff_syst = reader2.read_hist_1d('diff_err_tot')

hjs_b_pt = []
hjs_incl_pt=[]
hpy_b_pt = []
hpy_incl_pt=[]
for i in range(6):
    hjs_b_pt.append(reader.read_hist_1d('dr_signal_bjtc_jetShape_step3_{0}_0'.format(i)))
    hjs_incl_pt.append(reader.read_hist_1d('dr_signal_dijtc_jetShape_step3_{0}_0'.format(i)))
for i in range(5):
    hpy_b_pt.append(reader.read_hist_1d('dr_signal_bjtc_jetShape_step3_{0}_0'.format(i)))
    hpy_incl_pt.append(reader.read_hist_1d('dr_signal_dijtc_jetShape_step3_{0}_0'.format(i)))

In [3]:
## load data
# x-axis : dr 
ndigi = 3
dr = Variable("$\Delta r$", is_independent = True)
dr.values = round_pair(hjs_incl["x_edges"][:11],ndigi)

# y-axis : jet shapes

ptlabel = ["1<p_T^{\text{trk}}<2",
           "2<p_T^{\text{trk}}<3",
           "3<p_T^{\text{trk}}<4",
           "4<p_T^{\text{trk}}<8",
           "8<p_T^{\text{trk}}<12",
           "p_T^{\text{trk}} > 12",
          ]
js_incl_pt= []
js_incl_pt_err=[]
js_b_pt= []
js_b_pt_err=[]
for i in range(5):
    js_incl_pt.append(
        Variable("Charged particle yield of inclusive jets for ${ptlabel}$ GeV"
                 .format(ptlabel=ptlabel[i])
                 , is_independent = False, is_binned=False, units=""))
    js_incl_pt[i].values = round_list(hjs_incl_pt[i]["y"][:11],ndigi)
    js_incl_pt_err.append(Uncertainty("Statistical Uncert", is_symmetric = True))
    js_incl_pt_err[i].values = round_list(hjs_incl_pt[i]['dy'][:11],ndigi)
    js_incl_pt[i].add_uncertainty(js_incl_pt_err[i])
    
    js_b_pt.append(
        Variable("Charged particle yield of b jets for ${ptlabel}$ GeV"
                 .format(ptlabel=ptlabel[i])
                 , is_independent = False, is_binned=False, units=""))
    js_b_pt[i].values = round_list(hjs_b_pt[i]["y"][:11],ndigi)
    js_b_pt_err.append(Uncertainty("Statistical Uncert", is_symmetric = True))
    js_b_pt_err[i].values = round_list(hjs_b_pt[i]['dy'][:11],ndigi)
    js_b_pt[i].add_uncertainty(js_b_pt_err[i])
    
    
js_incl = Variable("Inclusive Jet Shapes", is_independent = False, is_binned=False, units="")
js_incl.values = round_list(hjs_incl["y"][:11],ndigi)

js_incl_err = Uncertainty("Statistical Uncert", is_symmetric = True)
js_incl_err.values = round_list(hjs_incl['dy'][:11],ndigi)
js_incl.add_uncertainty(js_incl_err)

js_incl_syst = Uncertainty("Systematic Uncert", is_symmetric = True)
js_incl_syst.values = round_list(hjs_incl_syst['dy'][:11],ndigi)
js_incl.add_uncertainty(js_incl_syst)

js_b = Variable("b Jet Shapes", is_independent = False, is_binned=False, units="")
js_b.values = round_list(hjs_b["y"][:11],ndigi)

js_b_syst = Uncertainty("Systematic Uncert", is_symmetric = True)
js_b_syst.values = round_list(hjs_b_syst['dy'][:11],ndigi)
js_b.add_uncertainty(js_b_syst)

js_b_err = Uncertainty("Statistical Uncert", is_symmetric = True)
js_b_err.values = round_list(hjs_b['dy'][:11],ndigi)
js_b.add_uncertainty(js_b_err)

ratio = Variable("b-to-Inclusive Jet Shape Ratios", is_independent = False, is_binned=False, units="")
ratio.values = round_list(hratio["y"][:11],ndigi)

ratio_syst = Uncertainty("Systematic Uncer", is_symmetric = True)
ratio_syst.values = round_list(hratio_syst['dy'][:11],ndigi)
ratio.add_uncertainty(ratio_syst)

ratio_err = Uncertainty("Statistical Uncert", is_symmetric = True)
ratio_err.values = round_list(hratio['dy'][:11],ndigi)
ratio.add_uncertainty(ratio_err)

# y-axis : particle yield
pt = [1,2,3,4,8, 12]
py_incl_pt= []
py_incl_pt_err=[]
py_b_pt= []
py_b_pt_err=[]
for i in range(5):
    py_incl_pt.append(
        Variable("Charged particle yield of inclusive jets for ${low}<p_T^{trk}<{high}$ GeV".format(low=pt[i],trk="trk",high=pt[i+1])
                 , is_independent = False, is_binned=False, units=""))
    py_incl_pt[i].values = round_list(hpy_incl_pt[i]["y"][:11],ndigi)
    py_incl_pt_err.append(Uncertainty("Statistical Uncert", is_symmetric = True))
    py_incl_pt_err[i].values = round_list(hpy_incl_pt[i]['dy'][:11],ndigi)
    py_incl_pt[i].add_uncertainty(py_incl_pt_err[i])
    
    py_b_pt.append(
        Variable("Charged particle yield of b jets for ${low}<p_T^{trk}<{high}$ GeV".format(trk="trk",low=pt[i],high=pt[i+1])
                 , is_independent = False, is_binned=False, units=""))
    py_b_pt[i].values = round_list(hpy_incl_pt[i]["y"][:11],ndigi)
    py_b_pt_err.append(Uncertainty("Statistical Uncert", is_symmetric = True))
    py_b_pt_err[i].values = round_list(hpy_incl_pt[i]['dy'][:11],ndigi)
    py_b_pt[i].add_uncertainty(py_incl_pt_err[i])

py_incl = Variable("Inclusive Charge Particle Yield", is_independent = False, is_binned=False, units="")
py_incl.values = round_list(hpy_incl["y"][:11],ndigi)

py_incl_err = Uncertainty("Statistical Uncert", is_symmetric = True)
py_incl_err.values = round_list(hpy_incl['dy'][:11],ndigi)
py_incl.add_uncertainty(py_incl_err)

py_incl_syst = Uncertainty("Systematic Uncert", is_symmetric = True)
py_incl_syst.values = round_list(hpy_incl_syst['dy'][:11],ndigi)
py_incl.add_uncertainty(py_incl_syst)

py_b = Variable("b Jet Charged Particle Yield", is_independent = False, is_binned=False, units="")
py_b.values = round_list(hpy_b["y"][:11],ndigi)

py_b_syst = Uncertainty("Systematic Uncert", is_symmetric = True)
py_b_syst.values = round_list(hpy_b_syst['dy'][:11],ndigi)
py_b.add_uncertainty(py_b_syst)

py_b_err = Uncertainty("Statistical Uncert", is_symmetric = True)
py_b_err.values = round_list(hpy_b['dy'][:11],ndigi)
py_b.add_uncertainty(py_b_err)

diff = Variable("Charged Particle Yield difference of b-to-Inclusive Jets", is_independent = False, is_binned=False, units="")
diff.values = round_list(hdiff["y"][:11],ndigi)

diff_syst = Uncertainty("Systematic Uncer", is_symmetric = True)
diff_syst.values = round_list(hdiff_syst['dy'][:11],ndigi)
diff.add_uncertainty(diff_syst)

diff_err = Uncertainty("Statistical Uncert", is_symmetric = True)
diff_err.values = round_list(hdiff['dy'][:11],ndigi)
diff.add_uncertainty(diff_err)

In [4]:
submission = Submission()
table1l = Table("Figure 1 (left)")
table1l.description = "The charged particle yield distribution $Y(\Delta r)$ of inclusive jets with $p_T > 120$ GeV and $1< p^{trk}_T < 16$ GeV are presented as functions of $\Delta r$ for differential $p_{T}^{trk}$ bin."
table1l.keywords["observation"]=["Charged Particle Yield"]
table1l.keywords["Kinematic Range"] = \
["jet pT >  120 GeV", "jet |eta|<1.6", "charged particles 1< pT < 12 GeV"]
table1l.add_variable(dr)
for h in py_incl_pt:
    table1l.add_variable(py_incl_pt[i])
submission.add_table(table1l)
    
table1r = Table("Figure 1 (right)")
table1r.description = "The charged particle yield distribution $Y(\Delta r)$ of b jets with $p_T > 120$ GeV and $1< p^{trk}_T < 16$ GeV are presented as functions of $\Delta r$ for differential $p_{T}^{trk}$ bin."
table1r.keywords["observation"]=["Charged Particle Yield"]
table1r.keywords["Kinematic Range"] = \
["jet pT >  120 GeV", "jet |eta|<1.6", "charged particles 1< pT < 12 GeV"]
table1r.add_variable(dr)
for h in py_b_pt:
    table1r.add_variable(py_b_pt[i])
submission.add_table(table1r)

In [5]:
table2l = Table("Figure 2 (left)")
table2l.description = "Charged particle yield distributions $Y(\Delta r)$ of inclusive jets with $1 < p_{\text{T}}^{\text{trk} < 12$ GeV are presented as functions of $\Delta r$.\
Inclusive jets with $p_T > 120$ GeV and charged particles with $1 < p^{\text{trk}_{\text{T}} < 12$ GeV are used to construct the distributions as functions of $\Delta r$ differential $p_T^{trk}$ bins."
table2l.keywords["observation"]=["Charged Particle Yield"]
table2l.keywords["Kinematic Range"] = ["jet pT >  120 GeV", "jet |eta|<1.6", "charged particles 1< pT < 12 GeV"]
table2l.add_variable(dr)
table2l.add_variable(py_incl)
submission.add_table(table2l)

table2m = Table("Figure 2 (middle)")
table2m.description = "Charged particle yield distributions $Y(\Delta r)$ of b jets with $1 < p_{\text{T}}^{\text{trk} < 12$ GeV are presented as functions of $\Delta r$.\
b jets with $p_T > 120$ GeV and charged particles with $1 < p^{\text{trk}_{\text{T}} < 12$ GeV are used to construct the distributions as functions of $\Delta r$ differential $p_T^{trk}$ bins."
table2m.keywords["observation"]=["Charged Particle Yield"]
table2m.keywords["Kinematic Range"] = ["jet pT >  120 GeV", "jet |eta|<1.6", "charged particles 1< pT < 12 GeV"]
table2m.add_variable(dr)
table2m.add_variable(py_b)
submission.add_table(table2m)

table2r = Table("Figure 2 (right)")
table2r.description = "Charged particle yield differences of b to inclusive jets for $1 < p_{\text{T}}^{\text{trk} < 12$ GeV are presented as functions of $\Delta r$.\
b jets with $p_T > 120$ GeV and charged particles with $1 < p^{\text{trk}_{\text{T}} < 12$ GeV are used to construct the distributions as functions of $\Delta r$ differential $p_T^{trk}$ bins."
table2r.keywords["observation"]=["Charged Particle Yield"]
table2r.keywords["Kinematic Range"] = ["jet pT >  120 GeV", "jet |eta|<1.6", "charged particles 1< pT < 12 GeV"]
table2r.add_variable(dr)
table2r.add_variable(diff)
submission.add_table(table2r)

In [6]:
table3l = Table("Figure 3 (left)")
table3l.description = "The jet shapes $\rho(\Delta r)$ of inclusive jets with $p_T > 120$ GeV and $p^{trk}_T > 1$ GeV are presented as functions of $\Delta r$ for differential $p_{T}^{trk}$ bin."
table3l.keywords["observation"]=["Jet Shapes"]
table3l.keywords["Kinematic Range"] = ["jet pT >  120 GeV", "jet |eta|<1.6", "charged particles pT > 1 GeV"]
table3l.add_variable(dr)
for h in js_incl_pt:
    table3l.add_variable(js_incl_pt[i])
submission.add_table(table3l)
    
table3r = Table("Figure 3 (right)")
table3r.description = "The jet shape distributions $\rho(\Delta r)$ of b jets with $p_T > 120$ GeV and $1< p^{trk}_T < 16$ GeV are presented as functions of $\Delta r$ for differential $p_{T}^{trk}$ bin."
table3r.keywords["observation"]=["Jet Shapes"]
table3r.keywords["Kinematic Range"] = ["jet pT >  120 GeV", "jet |eta|<1.6", "charged particles pT > 1 GeV"]
table3r.add_variable(dr)
for h in js_b_pt:
    table3r.add_variable(js_b_pt[i])
submission.add_table(table3r)

In [7]:
table4l = Table("Figure 4 Left")
table4l.description = "The jet shapes $\rho(\Delta r)$ of inclusive jets with $p_T > 120$ GeV and $p^{trk}_T > 1$ GeV are presented as functions of $\Delta."
table4l.keywords["observation"]=["Jet Shapes"]
table4l.keywords["Kinematic Range"] = \
["jet pT >  120 GeV", "jet |eta|<1.6", "charged particles pT > 1 GeV"]
table4l.add_variable(dr)
table4l.add_variable(js_incl)
submission.add_table(table4l)

table4m = Table("Figure 4 (middle)")
table4m.description = "The jet shapes $\rho(\Delta r)$ of b jets with $p_T > 120$ GeV and $p^{trk}_T > 1$ GeV are presented as functions of $\Delta."
table4m.keywords["observation"]=["Jet Shapes"]
table4m.keywords["Kinematic Range"] = \
["jet pT >  120 GeV", "jet |eta|<1.6", "charged particles pT > 1 GeV"]
table4m.add_variable(dr)
table4m.add_variable(js_b)
submission.add_table(table4m)

table4r = Table("Figure 4 (right)")
table4r.description = "The b-to-inclusive jet shape ratios for $p_T > 120$ GeV and $p^{trk}_T > 1$ GeV are presented as functions of $\Delta."
table4r.keywords["observation"]=["Jet Shapes"]
table4r.keywords["Kinematic Range"] = \
["jet pT >  120 GeV", "jet |eta|<1.6", "charged particles pT > 1 GeV"]
table4r.add_variable(dr)
table4r.add_variable(ratio)
submission.add_table(table4r)



In [8]:
submission.create_files("hin_18_020_hepdata")

In [9]:
!head hin_18_020_hepdata/figure_1_(right).yaml

/bin/bash: -c: line 0: syntax error near unexpected token `('
/bin/bash: -c: line 0: `head hin_18_020_hepdata/figure_1_(right).yaml'
